In [1]:
from data_utils import *


c:\Users\Jai\Desktop\BGP-WordEmbedding\BGP_venv\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Jai\Desktop\BGP-WordEmbedding\BGP_venv\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
c:\Users\Jai\Desktop\BGP-WordEmbedding\BGP_venv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
def num_dim_to_keep(total_var_explained, label):
    if label.endswith("8_dim"):
        return 8
    elif label.endswith("3_dim"):
        return 3
    else:
        return np.argmax(total_var_explained>0.9)


In [ ]:
# Do we even need df? Can we just save need probabilities?
df = Serialization.load_obj(f"stance_pipeline_full_data_{NUM_QUANTILES}_quantiles_full_data")
x = df.groupby("bin").mean()[FEATURE_COLUMNS]
df = get_sub_marker_pairs(df)
# Write unit test for below
bins, comms, markers, com_markers = get_bin_com_markers(df, bins, comms)
pav_matrix = Serialization.load_obj("pavalanathan_cooc_data_full_data")
cooc_matrix = Serialization.load_obj("our_cooc_data_full_data")


In [ ]:
matrices = [cooc_matrix, cooc_matrix, cooc_matrix, pav_matrix, pav_matrix, pav_matrix, pav_matrix]
labels = ["our_ppmi_3_dim", "our_raw_90_var", "our_ppmi_8_dim", "pav_raw_8_dim", "pav_ppmi_8_dim", "pav_raw_90_var", "pav_ppmi_90_var"]
delegates = [lambda x: create_derived_representation(x, include_ppmi=True), lambda x: create_derived_representation(x, include_ppmi=False), lambda x: create_derived_representation(x, include_ppmi=True), lambda x: create_derived_representation(x, include_ppmi=False), lambda x: create_derived_representation(x, include_ppmi=True), lambda x: create_derived_representation(x, include_ppmi=False), lambda x: create_derived_representation(x, include_ppmi=True)]

In [ ]:
all_data = {}
for matrix, label, delegate in zip(matrices, labels, delegates):
    curr_output = delegate(matrix)
    all_data[label] = curr_output

In [ ]:
for label in labels:
    eigenvalues, var_explained, total_var_explained = scree_plot(all_data[label]['singular_values'], to_show=False)
    all_data[label]['eigenvalues'] = eigenvalues
    all_data[label]['var_explained'] = var_explained
    all_data[label]['total_var_explained'] = total_var_explained
    all_data[label]['num_dim_to_keep'] = num_dim_to_keep(total_var_explained, label)
    all_data[label]['new_sem_rep'] = all_data[label]['sem_rep'][:, :all_data[label]['num_dim_to_keep']]
    all_data[label]['new_marker_rep'] = all_data[label]['marker_rep'].T[:, :all_data[label]['num_dim_to_keep']].reshape(-1, all_data[label]['num_dim_to_keep'])
    all_data[label]['new_sem_loadings'] = all_data[label]['sem_loadings'][:, :all_data[label]['num_dim_to_keep']]
    all_data[label]['new_marker_loadings'] = all_data[label]['marker_loadings'].T[:, :all_data[label]['num_dim_to_keep']].reshape(-1, all_data[label]['num_dim_to_keep'])


In [4]:
for label in labels:
    print(label)
    print(all_data[label]['num_dim_to_keep'])
    print(all_data[label]['total_var_explained'][all_data[label]['num_dim_to_keep']])

NameError: name 'labels' is not defined

In [ ]:
com_to_stance_usage = {"our_raw_90_var": {}, "our_ppmi_8_dim": {}}
comms = sorted(comms)
for i in range(len(comms)):
    sub = all_data['our_raw_90_var']['new_marker_loadings'][i*len(markers):(i+1)*len(markers), :]
    com_to_stance_usage['our_raw_90_var'][comms[i]] = sub
    sub = all_data['our_ppmi_8_dim']['new_marker_loadings'][i*len(markers):(i+1)*len(markers), :]
    com_to_stance_usage['our_ppmi_8_dim'][comms[i]] = sub
    # sub.columns = [col[col.index("_") + 1:] for col in sub.columns]
    

pav_comms = pav_matrix.index.tolist()
for label in tqdm(labels):
    # pav_full_vals = []
    # pav_partial_vals = []
    # our_full_vals = []
    # our_partial_vals = []
    full_vals = []
    partial_vals = []
    for i, row in crossposting_df.iterrows():
        com_1, com_2 = row['com_1'], row['com_2']
        if label.startswith("pav"):
            # Pav data
            ci1, ci2 = pav_comms.index(com_1), pav_comms.index(com_2)
            ci1_rep = all_data[label]['new_sem_loadings'][ci1].reshape(1, -1)
            ci2_rep = all_data[label]['new_sem_loadings'][ci2].reshape(1, -1)
            pav_full = cosine_similarity(ci1_rep, ci2_rep)[0][0]
            full_vals.append(pav_full)

            pav_partial = np.abs(ci1_rep - ci2_rep)[0]
            partial_vals.append(pav_partial)
        else:
            # Our Data
            full_vals.append(np.mean(cosine_similarity(com_to_stance_usage[label][com_1], com_to_stance_usage[label][com_2]).diagonal()))
            partial_vals.append(np.abs(com_to_stance_usage[label][com_1] - com_to_stance_usage[label][com_2]).sum(axis=0))
    all_data[label]['full_crosspost'] = full_vals
    all_data[label]['partial_crosspost'] = partial_vals




In [ ]:
NUM_CV = 5
L1_RATIO=0.5

    
df_data = []
for label in labels:
    lr = LogisticRegression(random_state=42, penalty='l2', solver="lbfgs")#, l1_ratio=L1_RATIO)
    full_scores = cross_val_score(lr, np.array(all_data[label]['full_crosspost']).reshape(-1, 1), crossposting_df['label'].tolist(), cv=NUM_CV)
    print(label)
    print("Full:%0.2f accuracy with a standard deviation of %0.2f" % (full_scores.mean(), full_scores.std()))

    lr = LogisticRegression(random_state=42, penalty='l2', solver="lbfgs", max_iter=100000)#, l1_ratio=L1_RATIO)
    partial_scores = cross_val_score(lr, np.array(all_data[label]['partial_crosspost']).reshape(-1, all_data[label]['num_dim_to_keep']), crossposting_df['label'].tolist(), cv=NUM_CV)
    print("Partial: %0.2f accuracy with a standard deviation of %0.2f" % (partial_scores.mean(), partial_scores.std()))
    print("\n")

    df_data.append([label, full_scores.mean(), partial_scores.mean()])